# Teaching with and Learning from Demonstration model
This notebook generates simulation data for four models: the doing demonstrator, showing model, naive observer, and sophisticated observer. Visualizations of these simulations are in the Visualization notebook.

In [1]:
import copy
from itertools import product
import time
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from mdp_lib.domains.gridworld import GridWorld
from planninginbeliefmodel import PlanningInObserverBeliefModel
from mdp_lib.domains.gridworldvis import visualize_trajectory, plot_text
from task import mdp_codes, mdp_params
from mdp_lib.util import sample_prob_dict
from util import mdp_to_feature_belief

np.random.seed(128374027)
np.seterr(under='ignore')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'raise'}

In [5]:
print(mdp_params)

[{'wait_action': False, 'gridworld_array': ['.oooo.', '.oppp.', '.opccy', '.oppc.', '.cccc.'], 'init_state': (0, 2), 'step_cost': 0, 'discount_rate': 0.99, 'feature_rewards': {'y': 50, 'p': 0, 'c': 0, 'o': 0, '.': 0}, 'absorbing_states': [(5, 2)], 'wall_action': False}, {'wait_action': False, 'gridworld_array': ['.oooo.', '.oppp.', '.opccy', '.oppc.', '.cccc.'], 'init_state': (0, 2), 'step_cost': 0, 'discount_rate': 0.99, 'feature_rewards': {'y': 50, 'p': 0, 'c': -10, 'o': 0, '.': 0}, 'absorbing_states': [(5, 2)], 'wall_action': False}, {'wait_action': False, 'gridworld_array': ['.oooo.', '.oppp.', '.opccy', '.oppc.', '.cccc.'], 'init_state': (0, 2), 'step_cost': 0, 'discount_rate': 0.99, 'feature_rewards': {'y': 50, 'p': -10, 'c': 0, 'o': 0, '.': 0}, 'absorbing_states': [(5, 2)], 'wall_action': False}, {'wait_action': False, 'gridworld_array': ['.oooo.', '.oppp.', '.opccy', '.oppc.', '.cccc.'], 'init_state': (0, 2), 'step_cost': 0, 'discount_rate': 0.99, 'feature_rewards': {'y': 50, '

# Parameters for doing and showing demonstration models

In [15]:
#doing model parameters
do_discount = .99
do_softmax = .08

#showing model parameters
base_discount_rate = .99
base_softmax_temp = 3
obmdp_discount_rate = .9
obmdp_softmax_temp= 1

# Doing model
This code builds doing models for the different MDPs that are defined in `task.py`. It also generates seed trajectories for building a discretized observer belief MDP (OBMDP) transition function. Using trajectories guided by what a doing agent would do helps focus the approximation on parts of the world-belief state space that are relevant.

In [16]:
doing_models = []
seed_trajs = []
for p in mdp_params:
    p = copy.deepcopy(p)
    p['discount_rate'] = do_discount
    m = GridWorld(**p)
    m.solve()
    doing_models.append(m)
    
    #generate seed trajectories for OBMDP discretization
    for _ in xrange(20):
        traj = m.run(temp=.7)
        if traj[-1][1] != '%':
            continue
        seed_trajs.append([(w, a) for w, a, _, _ in traj])
        
with open("./cached_values/seed_trajs.pkl", 'wb') as f:
    pickle.dump(seed_trajs, f)

# Showing model
This block builds showing models for the different possible OBMDPs using the `PlanningInObserverBeliefModel` class defined in `planninginbeliefmodel.py`. It discretizes the belief space using the `seed_trajs` generated in the previous block and solves the resulting (large) discrete MDP.

**Estimated running time: 6-8 mins**.

*Note that since the ground transition function is the same across all the ground MDPs considered here, the world-belief transitions are also the same. This means we can speed up computations slightly by reusing the transition function, `tf`.*

In [22]:
m = PlanningInObserverBeliefModel(
    base_discount_rate = do_discount,
    base_softmax_temp = do_softmax,
    obmdp_discount_rate = obmdp_discount_rate,
    obmdp_softmax_temp=obmdp_softmax_temp,
    true_mdp_code=mdp_codes[0],
    discretized_tf=None
)
m.seed_beliefs_with_trajs(seed_trajs)
m.build()
m.solve()
showing_models_inner_temp = m.ob_mdp

KeyboardInterrupt: 

In [23]:
showing_models = []
tf = None
for i, rfc in enumerate(mdp_codes):
    starttime = time.time()
    print rfc,
    m = PlanningInObserverBeliefModel(
        base_discount_rate = base_discount_rate,
        base_softmax_temp = base_softmax_temp,
        obmdp_discount_rate = obmdp_discount_rate,
        obmdp_softmax_temp=obmdp_softmax_temp,
        
        true_mdp_code=rfc,
        discretized_tf=tf
    )
    m.seed_beliefs_with_trajs(seed_trajs)
    m.build()
    m.solve()
    showing_models.append(m.ob_mdp)
    tf = m.ob_mdp.get_discretized_tf()
    print " %.2fs" % (time.time() - starttime)

ooo  35.71s
oox  38.17s
oxo  40.49s
oxx  46.82s
xoo  43.96s
xox  43.75s
xxo  44.92s
xxx  93.92s


# Generating trajectories and calculating observer beliefs

The following two blocks generate *doing* and *showing* world-state/action trajectories using the models defined above. For each sequence of world-states and actions, we then calculate the change in the observer models' beliefs over time. The two models are the *naive* and *sophisticated* observers, which correspond to the following equations, respectively:

\begin{align}
b^{\text{Obs}}_{t+1}(M_i) &= P(M_i \mid w_t, a_t, w_{t+1})\\
&\propto P(a_t, w_{t+1} \mid w_t, M_i)P(M_i)\\
&= P(a_t \mid w_t, M_i)P(w_{t+1} \mid w_t, a_t, M_i)P(M_i)\\
&= \pi_{i}^{\text{Do}}(a_t \mid w_t)T_{i}(w_{t+1} \mid w_t, a_t)b_t^{\text{Obs}}(M_i).
\end{align}

and

\begin{align}
b^{\text{S-Obs}}_{t+1}(M_i) &= P(M_i \mid w_t, b^{\text{Obs}}_t, a_t, w_{t+1}, b^{\text{Obs}}_{t+1}) \\
&\propto \pi_i^{\text{Show}}(a_t \mid w_t, b_t^{\text{Obs}})T_i(w_{t+1} \mid w_t, a_t)b_t^{\text{S-Obs}}(M_i).
\end{align}

Each trajectory and final belief state is recorded in the `model_obs_judgments` dataframe and cached.

**Estimated running time: 20 min**

In [7]:
def calc_obs_sobs_traj(wtraj):
    b_sobs = np.array(showing_models[0].get_init_state()[0])
    s = showing_models_inner_temp.get_init_state()
    inner_s = s
    
    obs_traj = [s[0],]
    sobs_traj = [b_sobs,]
    for w, a in wtraj:
        # get next naive belief
        ns = showing_models_inner_temp.transition(s=s, a=a)
        obs_traj.append(ns[0])
        
        # calc next sophisticated belief
        inner_ns = showing_models[0].transition(s=inner_s, a=a)
        show_a_probs = []
        for m in showing_models:
            a_probs = m.get_softmax_actionprobs(s=inner_s, temp=obmdp_softmax_temp)
            show_a_probs.append(a_probs[a])
        show_a_probs = np.array(show_a_probs)
        b_sobs = b_sobs*show_a_probs
        b_sobs = b_sobs/np.sum(b_sobs)
        sobs_traj.append(b_sobs)
        
        s = ns
        inner_s = inner_ns
    return {'obs_traj': obs_traj, 'sobs_traj': sobs_traj}

def is_correct(row):
    rf = dict(zip(['orange', 'purple', 'cyan'], row['rf']))
    if rf[row['color']] == 'x' \
            and row['exp_safe'] < .5:
        return True
    elif rf[row['color']] == 'o' \
            and row['exp_safe'] >= .5:
        return True
    return False

def calc_correct_prob(row):
    rf = dict(zip(['orange', 'purple', 'cyan'], row['rf']))
    if rf[row['color']] == 'x':
        return 1 - row['exp_safe']
    elif rf[row['color']] == 'o':
        return row['exp_safe']

In [8]:
n_trajs = 100
forder = ['orange', 'purple', 'cyan']
model_obs_judgments = []
for mi, (do_m, show_m) in enumerate(zip(doing_models, showing_models)):
    do_wtrajs = []
    show_wtrajs = []
    
    print mi,
    starttime = time.time()
    for _ in xrange(n_trajs):
        # generate and interpret DOING trajectory
        do_traj = do_m.run(temp=do_softmax)
        do_traj = [(w, a) for w, a, nw, r in do_traj]
        
        belief_trajs = calc_obs_sobs_traj(do_traj)
        obs_judg = mdp_to_feature_belief(belief_trajs['obs_traj'][-1], mdp_codes, forder)
        obs_judg['rf'] = mdp_codes[mi]
        obs_judg['observer'] = 'naive'
        obs_judg['demonstrator'] = 'doing'
        obs_judg['traj'] = do_traj
        obs_judg['belief_traj'] = belief_trajs['obs_traj']
        model_obs_judgments.append(obs_judg)
        
        sobs_judg = mdp_to_feature_belief(belief_trajs['sobs_traj'][-1], mdp_codes, forder)
        sobs_judg['rf'] = mdp_codes[mi]
        sobs_judg['observer'] = 'sophisticated'
        sobs_judg['demonstrator'] = 'doing'
        sobs_judg['traj'] = do_traj
        sobs_judg['belief_traj'] = belief_trajs['sobs_traj']
        model_obs_judgments.append(sobs_judg)
        
        # generate and interpret SHOWING trajectory
        show_traj = show_m.run(temp=obmdp_softmax_temp)
        show_traj = [(w, a) for (b, w), a, ns, r in show_traj]
        
        belief_trajs = calc_obs_sobs_traj(show_traj)
        obs_judg = mdp_to_feature_belief(belief_trajs['obs_traj'][-1], mdp_codes, forder)
        obs_judg['rf'] = mdp_codes[mi]
        obs_judg['observer'] = 'naive'
        obs_judg['demonstrator'] = 'showing'
        obs_judg['traj'] = show_traj
        obs_judg['belief_traj'] = belief_trajs['obs_traj']
        model_obs_judgments.append(obs_judg)
        
        sobs_judg = mdp_to_feature_belief(belief_trajs['sobs_traj'][-1], mdp_codes, forder)
        sobs_judg['rf'] = mdp_codes[mi]
        sobs_judg['observer'] = 'sophisticated'
        sobs_judg['demonstrator'] = 'showing'
        sobs_judg['traj'] = show_traj
        sobs_judg['belief_traj'] = belief_trajs['sobs_traj']
        model_obs_judgments.append(sobs_judg)
    print " %.2fs" % (time.time() - starttime)
        
model_obs_judgments = pd.DataFrame(model_obs_judgments)
model_obs_judgments = pd.melt(model_obs_judgments,
    id_vars=['demonstrator', 'rf', 'observer', 'traj', 'belief_traj'], 
    value_name='exp_safe', 
    var_name='color')

model_obs_judgments['confidence'] = model_obs_judgments['exp_safe'].apply(lambda v: abs(.5-v))
model_obs_judgments['correct'] = model_obs_judgments.apply(is_correct, axis=1)
model_obs_judgments['correct_prob'] = model_obs_judgments.apply(calc_correct_prob, axis=1)
model_obs_judgments.to_pickle('./cached_values/model_.pkl')

0  107.16s
1  105.14s
2  105.61s
3  106.44s
4  106.16s
5  2043.07s
6  1488.26s
7  107.68s


In [6]:
print(mdp_codes)

['ooo', 'oox', 'oxo', 'oxx', 'xoo', 'xox', 'xxo', 'xxx']


In [8]:
for mdp_code, mdp_param in zip(mdp_codes, mdp_params):
    print(mdp_code)
    print(mdp_param['feature_rewards'])

ooo
{'y': 50, 'p': 0, 'c': 0, 'o': 0, '.': 0}
oox
{'y': 50, 'p': 0, 'c': -10, 'o': 0, '.': 0}
oxo
{'y': 50, 'p': -10, 'c': 0, 'o': 0, '.': 0}
oxx
{'y': 50, 'p': -10, 'c': -10, 'o': 0, '.': 0}
xoo
{'y': 50, 'p': 0, 'c': 0, 'o': -10, '.': 0}
xox
{'y': 50, 'p': 0, 'c': -10, 'o': -10, '.': 0}
xxo
{'y': 50, 'p': -10, 'c': 0, 'o': -10, '.': 0}
xxx
{'y': 50, 'p': -10, 'c': -10, 'o': -10, '.': 0}


In [38]:
def get_human_traj_likelihood(human_trajs, traj_type='show'):
    """
    Calculates log likelihood of showing human trajs under doing and showing model.
    """
    ll_do, ll_show = 0, 0
    total_trajs = 0
    for rf, doing_model, showing_model in zip(mdp_codes, doing_models, showing_models):
        do_policy = doing_model.get_softmax_function(do_softmax)
        show_policy = showing_model.get_softmax_function(do_softmax)
        if traj_type == 'show':
            wtrajs = human_trajs[rf][1]
        elif traj_type == 'do':
            wtrajs = human_trajs[rf][0]
        total_trajs += len(wtrajs)
        for wtraj in wtrajs:
            s = showing_model.get_init_state()
            for w, a in wtraj:
                ll_do += np.log(do_policy[w][a])
                show_action_prob = showing_model.get_softmax_actionprobs(
                    s=s, temp=obmdp_softmax_temp)[a]
                ll_show += np.log(show_action_prob)
                s = showing_model.transition(s=s, a=a)
    ll_do, ll_show = ll_do/total_trajs, ll_show/total_trajs
    return ll_do, ll_show

In [12]:
human_trajs = get_human_trajs()

In [33]:
get_human_traj_likelihood(human_trajs)

(-42.99551940614264, -26.92586380495133)

In [37]:
get_human_traj_likelihood(human_trajs)

(-21.11288195803672, -28.0774046726638)

In [3]:
def get_human_trajs():
    trials = pd.read_pickle('./data/teaching-trials.pd.pkl')
    trials = trials[trials['phase'] == 'demonstration']
    do_trajs, show_trajs = [], []
    rf_order = ['ooo', 'xoo', 'oox', 'oxo', 'xox', 'xxo', 'oxx', 'xxx']
    cond_order = ['do', 'show']
    trajs = {}
    for rf in rf_order:
        trajs[rf] = [[], []]
    for (rf, cond, p), ptrials in trials.groupby(['rf', 'cond', 'participant']):
        ptrials = ptrials.sort_values('timestep')
        traj = list(ptrials['stateaction'])# + [((5, 2), 'x'), ]
        if cond == 'do':
            trajs[rf][0].append(traj)
        elif cond == 'show':
            trajs[rf][1].append(traj)
    return trajs

def get_human_results():
    wtrajs = get_human_trajs()
    forder = ['orange', 'purple', 'cyan']
    model_obs_judgments = []
    for mi, (do_m, show_m) in enumerate(zip(doing_models, showing_models)):
        do_wtrajs = []
        show_wtrajs = []

        rf = mdp_codes[mi]
        print mi,
        starttime = time.time()
        human_trajs = wtrajs[rf]
        for (do_traj, show_traj) in zip(human_trajs[0], human_trajs[1]):
            # interpret DOING trajectory
            belief_trajs = calc_obs_sobs_traj(do_traj)
            obs_judg = mdp_to_feature_belief(belief_trajs['obs_traj'][-1], mdp_codes, forder)
            obs_judg['rf'] = rf
            obs_judg['observer'] = 'naive'
            obs_judg['demonstrator'] = 'doing'
            obs_judg['traj'] = do_traj
            obs_judg['belief_traj'] = belief_trajs['obs_traj']
            model_obs_judgments.append(obs_judg)

            sobs_judg = mdp_to_feature_belief(belief_trajs['sobs_traj'][-1], mdp_codes, forder)
            sobs_judg['rf'] = rf
            sobs_judg['observer'] = 'sophisticated'
            sobs_judg['demonstrator'] = 'doing'
            sobs_judg['traj'] = do_traj
            sobs_judg['belief_traj'] = belief_trajs['sobs_traj']
            model_obs_judgments.append(sobs_judg)

            # interpret SHOWING trajectory
            belief_trajs = calc_obs_sobs_traj(show_traj)
            obs_judg = mdp_to_feature_belief(belief_trajs['obs_traj'][-1], mdp_codes, forder)
            obs_judg['rf'] = rf
            obs_judg['observer'] = 'naive'
            obs_judg['demonstrator'] = 'showing'
            obs_judg['traj'] = show_traj
            obs_judg['belief_traj'] = belief_trajs['obs_traj']
            model_obs_judgments.append(obs_judg)

            sobs_judg = mdp_to_feature_belief(belief_trajs['sobs_traj'][-1], mdp_codes, forder)
            sobs_judg['rf'] = rf
            sobs_judg['observer'] = 'sophisticated'
            sobs_judg['demonstrator'] = 'showing'
            sobs_judg['traj'] = show_traj
            sobs_judg['belief_traj'] = belief_trajs['sobs_traj']
            model_obs_judgments.append(sobs_judg)
        print " %.2fs" % (time.time() - starttime)

    model_obs_judgments = pd.DataFrame(model_obs_judgments)
    model_obs_judgments = pd.melt(model_obs_judgments,
        id_vars=['demonstrator', 'rf', 'observer', 'traj', 'belief_traj'], 
        value_name='exp_safe', 
        var_name='color')

    model_obs_judgments['confidence'] = model_obs_judgments['exp_safe'].apply(lambda v: abs(.5-v))
    model_obs_judgments['correct'] = model_obs_judgments.apply(is_correct, axis=1)
    model_obs_judgments['correct_prob'] = model_obs_judgments.apply(calc_correct_prob, axis=1)
    model_obs_judgments.to_pickle('./cached_values/human_results.pkl')

In [31]:
get_human_results()

0  0.11s
1  0.15s
2  0.16s
3  0.16s
4  0.14s
5  0.14s
6  0.12s
7  0.10s
